In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from category_encoders import TargetEncoder
from category_encoders import MEstimateEncoder
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from sklearn.impute import KNNImputer
from sklearn.metrics import make_scorer
import datetime as dt
from datetime import timedelta
from sklearn import metrics
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.special import boxcox,inv_boxcox
from sklearn.feature_selection import SelectFromModel

import warnings
warnings.filterwarnings('ignore')

In [16]:
train = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Predicting_Churn_Rate/train.csv')
test = pd.read_csv('C:/Users/DELL/Desktop/HackerEarth_Predicting_Churn_Rate/test.csv')

In [17]:
train['churn_risk_score'].value_counts()

 3    10424
 4    10185
 5     9827
 2     2741
 1     2652
-1     1163
Name: churn_risk_score, dtype: int64

In [22]:
train[(train['churn_risk_score'] == -1) & (train['avg_time_spent'] < 0)].shape
# train[train['avg_time_spent'] < 0].shape

(60, 25)

In [3]:
t = test['customer_id']
train['churn_risk_score'] = train['churn_risk_score'].abs()
# # a = train[train['churn_risk_score'] == -1]
# # b = train[train['churn_risk_score'] != -1]

In [4]:
test['churn_risk_score'] = -1
data = pd.concat([train,test],axis = 0)

Identifying hidden NAN values

In [5]:
data['joined_through_referral'] = data['joined_through_referral'].replace('?',np.nan)
data['medium_of_operation'] = data['medium_of_operation'].replace('?',np.nan)
data['avg_frequency_login_days'] = data['avg_frequency_login_days'].replace('Error',np.nan)

Applying mod to negative values

In [6]:
data['days_since_last_login'] = data['days_since_last_login'].abs()
data['avg_time_spent'] = data['avg_time_spent'].abs()
data['points_in_wallet'] = data['points_in_wallet'].abs()

data['avg_frequency_login_days'] = data['avg_frequency_login_days'].astype(float)
data['avg_frequency_login_days'] = data['avg_frequency_login_days'].abs()

Extracting dates

In [7]:
data['joining_date'] = pd.to_datetime(data['joining_date'],infer_datetime_format = True)
data['last_visit_time'] = pd.to_datetime(data['last_visit_time'],format = '%H:%M:%S').dt.hour

In [8]:
a = data['joining_date'].max()
data['days_since_joined'] = a - data['joining_date']
data['days_since_joined'] = data['days_since_joined'].dt.days
data = data.drop(['Name','customer_id','security_no','referral_id','joining_date'],axis=1)

Removing missing values

In [9]:
data.fillna({'region_category':data['region_category'].mode()[0]\
              ,'joined_through_referral':data['joined_through_referral'].mode()[0],'preferred_offer_types':data['preferred_offer_types'].mode()[0]\
              ,'medium_of_operation':data['medium_of_operation'].mode()[0]}, inplace=True)


data.fillna({'points_in_wallet':data['points_in_wallet'].mean(),'avg_frequency_login_days':data['avg_frequency_login_days'].mean()},inplace = True)

In [10]:
categorical = data.select_dtypes('object').columns
for col in categorical:
    data[col] = data[col].astype(str)

In [11]:
data['days_since_last_login'] = data['days_since_last_login'].replace(999,30)

In [12]:
data['avg_login_days'] = data['days_since_joined'] / data['avg_frequency_login_days']

In [13]:
data['x'] = data['days_since_last_login'] - data['avg_login_days']
# # le = LabelEncoder()
# # data['x'] = le.fit_transform(data[['x']])

In [14]:
# data['x'].value_counts()

Feature engineering

In [ ]:
# label_ranges = [i for i in range(0,24,4)]
# label_names = [i for i in range(5)]
# data['last_visit_time'] = pd.cut(data['last_visit_time'], bins = label_ranges, labels = label_names)

In [ ]:
# num = ['avg_transaction_value','points_in_wallet','avg_frequency_login_days','avg_time_spent']
# numdata = data[num]
# trans = PolynomialFeatures(degree=2)
# numdata = trans.fit_transform(numdata)
# numdata = pd.DataFrame(numdata)
# #for i in list(numdata.columns):
# #    numdata[i] = np.log(numdata[i])
# #    numdata[i] = numdata[i].round(1)


In [ ]:
# data.reset_index(drop=True, inplace=True)
# numdata.reset_index(drop=True, inplace=True)
# data = pd.concat([data,numdata],axis = 1)
# data = data.join(numdata)

In [ ]:
# from scipy.cluster.vq import kmeans, vq
# cluster_centers,_ = kmeans(data['avg_time_spent'],3)
# data['ats_labels'], _ = vq(data['avg_time_spent'],cluster_centers)

# cluster_centers,_ = kmeans(data['avg_transaction_value'],4)
# data['atv_labels'], _ = vq(data['avg_transaction_value'],cluster_centers)

# cluster_centers,_ = kmeans(data['points_in_wallet'],3)
# data['piw_labels'], _ = vq(data['points_in_wallet'],cluster_centers)

In [ ]:
# data['combo3'] = data['ats_labels'] + data['atv_labels'] + data['piw_labels']

In [ ]:
# def make_mi_scores(x, y):
#     mi_scores = mutual_info_classif(x, y)
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores

# mi_scores = make_mi_scores(x, y)
# imp_columns = list(mi_scores.index[:])
# x = x[imp_columns]
# test = test[imp_columns]
# print(mi_scores)

In [ ]:
# distortions = []
# num_clusters = range(2,7)
# for i in num_clusters:
#     centroids, distortion = kmeans(data['points_in_wallet'],i)
#     distortions.append(distortion)

# elbow = pd.DataFrame({'num_clusters':num_clusters,'distortions':distortions})
# elbow.plot(kind = 'line',x = 'num_clusters', y = 'distortions')  

In [ ]:
cluster_centers,_ = kmeans(data['days_since_last_login'],5)
data['dsll'], _ = vq(data['days_since_last_login'],cluster_centers)
data['dsll'] = data['dsll'].astype(str)

# cluster_centers,_ = kmeans(data['age'],4)
# data['age1'], _ = vq(data['age'],cluster_centers)
# data['ag1'] = data['age1'].astype(str)

cluster_centers,_ = kmeans(data['days_since_joined'],4)
data['dsj'], _ = vq(data['days_since_joined'],cluster_centers)
data['dsj'] = data['dsj'].astype(str)

cluster_centers,_ = kmeans(data['avg_frequency_login_days'],3)
data['ald'], _ = vq(data['avg_frequency_login_days'],cluster_centers)
data['ald'] = data['ald'].astype(str)

cluster_centers,_ = kmeans(data['last_visit_time'],4)
data['lvt'], _ = vq(data['last_visit_time'],cluster_centers)
data['lvt'] = data['lvt'].astype(str)


In [ ]:
# categorical = data.select_dtypes('object').columns
# combo = combinations(categorical,3) # 2 degree combinations
# for i, j,k in combo:
#     data[i + '_' + j + '_' + k] = data[i].astype(str) + '_' + data[j].astype(str) + '_' + data[k].astype(str)

In [14]:
data['medium_of_operation'].value_counts()

Desktop       29733
Smartphone    21281
Both           5897
Name: medium_of_operation, dtype: int64

In [15]:
data[data['days_since_last_login'] == 30].shape

(3021, 23)

In [54]:
train['churn_risk_score'].value_counts('normalize = True')

3    0.281791
4    0.275330
5    0.265652
1    0.103130
2    0.074097
Name: churn_risk_score, dtype: float64

In [57]:
train[train['internet_option'] == 'Fiber_Optic']['churn_risk_score'].value_counts()

4    3409
3    3403
5    3258
1    1278
2     888
Name: churn_risk_score, dtype: int64

In [56]:
test['internet_option'].value_counts()

Fiber_Optic    6747
Wi-Fi          6599
Mobile_Data    6573
Name: internet_option, dtype: int64

In [53]:
pd.crosstab(data['churn_risk_score'],data['internet_option'])

internet_option,Fiber_Optic,Mobile_Data,Wi-Fi
churn_risk_score,,,
-1,6747,6573,6599
1,1278,1292,1245
2,888,918,935
3,3403,3438,3583
4,3409,3372,3404
5,3258,3323,3246


In [15]:
data['feedback'].value_counts()

Poor Product Quality        9724
No reason specified         9713
Too many ads                9680
Poor Customer Service       9662
Poor Website                9572
Reasonable Price            2205
Products always in Stock    2160
Quality Customer Care       2115
User Friendly Website       2080
Name: feedback, dtype: int64

In [16]:
pd.crosstab(data['churn_risk_score'],data['feedback'])

feedback,No reason specified,Poor Customer Service,Poor Product Quality,Poor Website,Products always in Stock,Quality Customer Care,Reasonable Price,Too many ads,User Friendly Website
churn_risk_score,,,,,,,,,
-1,3423,3410,3374,3301,778,755,788,3401,689
1,218,196,198,211,715,666,732,183,696
2,0,0,0,0,667,694,685,0,695
3,2078,2064,2025,2098,0,0,0,2159,0
4,1996,2040,2120,2061,0,0,0,1968,0
5,1998,1952,2007,1901,0,0,0,1969,0


In [17]:
data['complaint_status'].value_counts()

Not Applicable              28552
Solved in Follow-up          7167
No Information Available     7100
Unsolved                     7096
Solved                       6996
Name: complaint_status, dtype: int64

In [58]:
data['past_complaint'].value_counts()

No     28552
Yes    28359
Name: past_complaint, dtype: int64

In [18]:
z = data[data['complaint_status'] == 'Unsolved']
z = z[(z['complaint_status'] != 'Products always in Stock') & (z['complaint_status'] != 'Quality Customer Care') & (z['complaint_status'] != 'Reasonable Price') & (z['complaint_status'] != 'Too many ads') & (z['complaint_status'] != 'User Friendly Website')]

In [19]:
data['churn_risk_score'].value_counts(normalize = True)

-1    0.350003
 3    0.183163
 4    0.178964
 5    0.172673
 1    0.067034
 2    0.048163
Name: churn_risk_score, dtype: float64

In [20]:
z['churn_risk_score'].value_counts(normalize = True)

-1    0.345547
 3    0.187289
 4    0.176015
 5    0.175028
 1    0.068207
 2    0.047914
Name: churn_risk_score, dtype: float64

In [21]:
pd.crosstab(z['complaint_status'],z['churn_risk_score'])

churn_risk_score,-1,1,2,3,4,5
complaint_status,,,,,,
Unsolved,2452,484,340,1329,1249,1242


In [22]:
# mask = (data['complaint_status'].isin['Unsolved']) & !(data['feedback'].isin['Products always in Stock','Quality Customer Care','Reasonable Price','Too many ads','User Friednly Website'])


Checking the distribution of data

In [23]:
col_to_normalize = ['avg_time_spent']
for i in col_to_normalize:
    data[i] = np.log(data[i])
    data[i] = data[i].round(1)

In [24]:
# cor = data.corr()
# sns.heatmap(cor,annot = True)

Polynomial combinations of numerical features

In [25]:
# trans = PolynomialFeatures(degree=2)
# data = trans.fit_transform(data)

Combining categorical features

In [26]:
# categorical = data.select_dtypes('object').columns
# combo = combinations(categorical,2) # 2 degree combinations
# for i, j in combo:
#     data[i + '_' + j] = data[i].astype(str) + '_' + data[j].astype(str)

Encoding

In [27]:
categorical = list(data.select_dtypes('object').columns)
le = LabelEncoder()
for col in categorical:
    data[col] = le.fit_transform(data[[col]])


In [28]:
test = data[data['churn_risk_score']== -1]
train = data[data['churn_risk_score']!= -1]
test = test.drop(['churn_risk_score'],axis=1)

In [29]:
y = train['churn_risk_score']
x = train.drop(['churn_risk_score'],axis=1)

In [30]:
x.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,...,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,days_since_joined,avg_login_days,x
0,18,0,2,3,0,1,1,2,16,17,...,17.0,781.750000,1,1,0,1,4,136,8.000000,9.000000
1,32,0,0,4,1,1,1,1,12,16,...,10.0,687.771646,1,0,1,2,5,125,12.500000,3.500000
2,44,0,1,2,1,1,1,2,22,14,...,22.0,500.690000,0,1,1,3,3,415,18.863636,-4.863636
3,37,1,0,2,1,1,1,1,15,11,...,6.0,567.660000,0,1,1,4,3,428,71.333333,-60.333333
4,31,0,0,2,0,0,2,1,15,20,...,16.0,663.060000,0,1,1,2,3,110,6.875000,13.125000


Target Encoding

In [31]:
encoder = MEstimateEncoder(return_df = True)
encoder.fit(x,y)
x = encoder.transform(x)
test = encoder.transform(test)

Calculating MI Scores

In [32]:
def make_mi_scores(x, y):
    mi_scores = mutual_info_classif(x, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=x.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(x, y)
imp_columns = list(mi_scores.index[:])
x = x[imp_columns]
test = test[imp_columns]
print(mi_scores)

points_in_wallet                0.571323
membership_category             0.545356
feedback                        0.357704
avg_transaction_value           0.153804
avg_frequency_login_days        0.080514
x                               0.024280
avg_login_days                  0.020765
days_since_last_login           0.005359
past_complaint                  0.005019
gender                          0.004542
region_category                 0.004264
joined_through_referral         0.003016
internet_option                 0.002560
age                             0.002196
complaint_status                0.002194
preferred_offer_types           0.001449
used_special_discount           0.000207
last_visit_time                 0.000000
medium_of_operation             0.000000
offer_application_preference    0.000000
days_since_joined               0.000000
avg_time_spent                  0.000000
Name: MI Scores, dtype: float64


In [33]:
def score(y_true, y_pred):
    score = 100 * metrics.f1_score(y_true, y_pred, average="macro")
    return score

Cross val score

In [34]:
cbc = CatBoostClassifier(verbose=0,max_depth = 6)
rfc = RandomForestClassifier(verbose = 0,n_jobs = -1,max_depth = 6)
xgbc = XGBClassifier(verbose = 0,n_jobs = -1,max_depth = 6)

In [35]:
# cv = cross_val_score(xgbc, x,y, scoring = make_scorer(score,greater_is_better = True) ,cv = 5)
# print(cv)
# print(np.mean(cv))

Final Submission

In [36]:
xgbc.fit(x,y)
pred = xgbc.predict(test)
test['customer_id']=t 
test['churn_risk_score'] = pred

test[['customer_id','churn_risk_score']].to_csv('submission22.csv',index=False)

In [ ]:
from xgboost import plot_importance
plot_importance(xgbc)

Using Regression

In [41]:
# from xgboost import XGBRegressor
# xgbr = XGBRegressor(verbose = 0,n_jobs = -1,max_depth = 6)
# xgbr.fit(x,y)
# pred = xgbr.predict(test)
# pred = pred.round()
# test['customer_id']=t 
# test['churn_risk_score'] = pred

# test[['customer_id','churn_risk_score']].to_csv('submission21.csv',index=False)

[11:08:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




Final Submission with feature selection using SelectFromModel

In [ ]:
# selector = SelectFromModel(estimator = ,prefit = True,max_features = , threshold = -np.inf)
# .fit(x,y)
# x = selector.transform(x)
# test = selector.transform(test)
# .fit(x,y)
# pred = .predict(test)
# test[''] = pred
# test[[]].to_csv('submission1.csv',index = False)

In [ ]:
# x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,test_size = 0.20, shuffle = True,random_state = 22,stratify = y)